In [ ]:
from tqdm import tqdm
from array_lib import *
from point3d_lib import Point
from skeleton3d_lib import Skeleton
from scipy.interpolate import CubicSpline
from skimage.morphology import skeletonize
from ply_creation_lib import create_ply, create_ply_normals
import matplotlib.pyplot as plt
import pydicom as dicom
import numpy as np
import itertools
import struct
import pickle
import time
import copy
import os

In [ ]:
"""
input_folder - must contain dcm files directly within it and only of one scan.
offsets - range of colors to include
seed - the entry coordinate to the aorta, must be the center.
threshold - normalized image color lower threshold
image_center - point between the arteries of interest
"""
input_folder = f'{os.getcwd()}\\20240923'
offsets = (-15, 20)
seed = (20, 310, 160)
image_center = (150, 280, 230)
min_skeleton_length = 100

In [ ]:
def read_dicom(input_folder: str) -> np.ndarray:
    files: list[str] = os.listdir(input_folder)
    data = [dicom.dcmread(f'{input_folder}\\{file}') for file in files if file.endswith('.dcm')]
    image = np.array([dicom.pixel_array(datum) for datum in data])
    return image

def normalize_image_colors(image: np.ndarray) -> np.ndarray:
    min_val = np.min(image)
    max_val = np.max(image)
    image = (image - min_val) / (max_val - min_val) * 255
    return image

image = normalize_image_colors(read_dicom(input_folder))

In [ ]:
filtered_mask = custom_floodfill_3d(image, seed_point=seed, new_value=-1, offsets=offsets)
eroded_mask = erode_3d(filtered_mask)
heartless_mask = remove_heart(eroded_mask)
trimmed_mask = distinguish_3d(heartless_mask)

In [ ]:
skeleton_mask = skeletonize(trimmed_mask)
closest_skeletons = find_closest_skeletons(skeleton_mask, image_center)
skeleton_points = np.concatenate([i[0] for i in closest_skeletons])
filtered_skeleton_mask = np.zeros_like(skeleton_mask).astype(bool)
filtered_skeleton_mask[skeleton_points[:, 0], skeleton_points[:, 1], skeleton_points[:, 2]] = True

selected_skeletons = floodfill_nearby_skeletons(heartless_mask, closest_skeletons)

In [ ]:
skeletons = [s[0] for s in closest_skeletons]

left_skel = Skeleton('left')
left_skel.add_points(skeletons[0], filtered_skeleton_mask)

right_skel = Skeleton('right')
right_skel.add_points(skeletons[1], filtered_skeleton_mask)

In [ ]:
left_skel.remove_close_ends()
right_skel.remove_close_ends()

In [ ]:
center_point = Point(image_center)
left_skel.find_head_point(center_point)
right_skel.find_head_point(center_point)

In [ ]:
branches: list[Skeleton] = []
for skel in [left_skel, right_skel]:
    branches.extend(skel.create_new_path_skeletons(min_skeleton_length))

In [ ]:
for i, branch in enumerate(branches):
    branch.calculate_normals()
    print(f'{i}) {len(branch)}')
    # create_ply_normals(branch, f'normals_{i+1}.ply')

In [ ]:
def interpolate_skeleton(old_skeleton: Skeleton, step=0.1) -> Skeleton:
    skeleton = smoothe_skeleton(copy.deepcopy(old_skeleton))
    points = np.array([p.coordinates for p in skeleton])
    cs_x = CubicSpline(range(len(points)), points[:, 0])
    cs_y = CubicSpline(range(len(points)), points[:, 1])
    cs_z = CubicSpline(range(len(points)), points[:, 2])
    
    new_parameter_values = np.linspace(0, len(points)-1, int(len(points) / step))
    new_x = cs_x(new_parameter_values)
    new_y = cs_y(new_parameter_values)
    new_z = cs_z(new_parameter_values)
    
    new_values = np.vstack([new_x, new_y, new_z]).T
    skeleton.points = [Point(val) for val in new_values]
    return skeleton

def smoothe_skeleton(skeleton: Skeleton) -> Skeleton:
    old_skeleton = copy.deepcopy(skeleton)
    smoothe = lambda arr, i: (arr[i-1].coordinates+arr[i].coordinates+arr[i+1].coordinates)/3
    for i in range(1, len(skeleton) - 1):
        skeleton[i].coordinates = smoothe(old_skeleton, i)
    return skeleton

interpolate_skeleton(branches[-1], step=0.1)

In [ ]:
def get_value(image: np.ndarray, coords: np.ndarray):
    x_size, y_size, z_size = image.shape
    coords = np.clip(coords, [0, 0, 0], [x_size-1, y_size-1, z_size-1])
    result = image[coords[:, 0], coords[:, 1], coords[:, 2]]
    return result

def mpr(image: np.ndarray, skeleton: Skeleton, rotations: int, x_step: int, y_step: int):
    middle_point: Point = copy.deepcopy(skeleton[len(skeleton)//2])
    main_skeleton = interpolate_skeleton(copy.deepcopy(skeleton), step=x_step)
    
    
    
    rotation_degrees = 360 // rotations
    for _ in range(rotations):
        
        skeleton = copy.deepcopy(main_skeleton)
        new_image = np.zeros((len(np.arange(-200, 201, y_step)), len(skeleton)))
        middle_point.rotate_normal(np.pi/180*rotation_degrees)
        coord_offset = middle_point.coordinates.copy()
        normal = middle_point.normal
        
        #1) calculate all the points coords relative to the middle point
        for point in skeleton:
            point.coordinates -= coord_offset
        
        #2) project all points to the normal line - this will be the x direction of the point from the center in a 2D image
        n_dot_n = np.dot(normal, normal)
        for point in skeleton:
            # magnitude of projection of point p = np.dot(p, n)/np.dot(n, n)
            point.projection_magnitude = np.dot(point.coordinates, normal) / n_dot_n
        
        #3) for the y direction - place the normal vector on each other point and calculate the values of the line
        for i, point in enumerate(skeleton):
            steps = np.arange(-200, 201, y_step).astype(float) + point.projection_magnitude
            coords = (point.coordinates + coord_offset + steps[:, np.newaxis] * normal + 0.5).astype(int)
            values = get_value(image, coords).astype(int)
            new_image[:, i] = values
        
        fig, axes = plt.subplots(1, 1, figsize=(15, 5))
        axes.imshow(new_image, cmap='gray')
        fig.suptitle(branch.name)
        plt.show()
    
    
    

for branch in branches:
    mpr(image, branch, rotations=1, x_step = 0.1, y_step = 0.1)

In [ ]:
def try_get_points(image: np.ndarray, points: list[np.ndarray]):
    w, h, d = image.shape
    x, y, z = points[:, 0], points[:, 1], points[:, 2]
    in_bounds = (0 <= x) & (x < w) & (0 <= y) & (y < h) & (0 <= z) & (z < d)
    values = np.zeros(len(points), dtype=image.dtype)
    valid_indices = np.where(in_bounds)[0]
    values[valid_indices] = image[x[in_bounds], y[in_bounds], z[in_bounds]]
    return values

def calculate_mpr(image: np.ndarray, skeleton: Skeleton):
    testing_mpr = np.zeros((201, len(skeleton)))
    for i, point in enumerate(skeleton):
        k = np.arange(-100, 101)
        vector_offsets = k[:, np.newaxis] * point.top_normal
        coords = (point.coordinates + vector_offsets).astype(int)
        new_values = try_get_points(image, coords)
        testing_mpr[:, i] = new_values
        plt.imshow(testing_mpr, cmap='gray')


for branch in branches[1:]:
    for point in branch:
        point.top_normal = np.array([0.0, 0.0, 1.0])
    calculate_mpr(image, branch)
    break
# for branch in branches:
#     
    
    
#     fig, axes = plt.subplots(1, , figsize=(15, 5))
#     axes[0].imshow(testing_mpr, cmap='gray')
#     fig.suptitle(branch.name)
#     plt.show()
#     plt.imshow(testing_mpr, cmap='gray')

In [ ]:
rotation_degrees = 45
number_of_rotations = 360 // rotation_degrees
for rot in range(number_of_rotations):
    degrees = rotation_degrees*rot
    
    for point in branch:
        point.rotate_normal(np.pi/180*degrees)
    create_ply_normals(branch, f'normals_{i+1}_rot{round(degrees, 2)}.ply')

In [ ]:
def display_branches(branches: list[Skeleton]) -> None:
    for branch in branches:
        branch_mask = np.zeros_like(image).astype(bool)
        for point in branch.points:
            branch_mask[tuple(point.coordinates)] = True
        create_ply(branch_mask, f'{branch.name}_branch_{branch.id}.ply')
        
display_branches(branches)

In [ ]:
def display_mpr(branch: Skeleton):
    mpr_test_x = np.zeros((image.shape[0], len(branch)))
    mpr_test_y = np.zeros((image.shape[1], len(branch)))
    mpr_test_z = np.zeros((image.shape[2], len(branch)))

    for i, p in enumerate(branch.points):
        coords = p.coordinates
        x_pixels = image[:, coords[1], coords[2]]
        y_pixels = image[coords[0], :, coords[2]]
        z_pixels = image[coords[0], coords[1], :]
        
        mpr_test_x[:, i] = x_pixels
        mpr_test_y[:, i] = y_pixels
        mpr_test_z[:, i] = z_pixels

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(mpr_test_x, cmap='gray')
    axes[1].imshow(mpr_test_y, cmap='gray')
    axes[2].imshow(mpr_test_z, cmap='gray')
    fig.suptitle(branch.name)
    plt.show()
    
for branch in branches:
    display_mpr(branch)

In [ ]:
create_ply(image, f'1.0_image.ply')
create_ply(filtered_mask, f'1.1_filtered.ply')
create_ply(eroded_mask, f'1.2_eroded.ply')
create_ply(heartless_mask, f'1.3_heartless.ply')
create_ply(trimmed_mask, f'1.4_trimmed.ply')
create_ply(skeleton_mask, f'1.5_skeleton.ply')
create_ply(filtered_skeleton_mask, f'1.6_closest_skeletons.ply')